In [88]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,precision_score,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from notifyme import notify
from tqdm import tqdm

In [2]:
df = pd.read_csv("./train_clean.csv")

In [3]:
df['comment_text'] = df.comment_text.fillna(value="spam")

In [4]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edit username hardcore metallica f...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww match background colour be seemingly stic...,0,0,0,0,0,0
2,000113f07ec002fd,hey man be try edit war guy constantly remove ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,not real suggestion improvement wonder section...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page s,0,0,0,0,0,0


In [5]:
def score(y,y_pred):
    print (f"Precision : {precision_score(y,y_pred)}")
    print (f"Recall : {recall_score(y,y_pred)}")
    print (f"Accuracy : {accuracy_score(y,y_pred)}")

In [6]:
features = df.comment_text.values

In [7]:
from collections import Counter

In [8]:
words = Counter()

In [9]:
for comment in df.comment_text.values:
    for word in comment.split(" "):
        words[word] += 1

In [10]:
features = df.pop("comment_text")

In [11]:
_ = df.pop("id")

In [12]:
cv = CountVectorizer()

In [13]:
features_vect = cv.fit_transform(features)

In [14]:
X,x,Y,y = train_test_split(features_vect,df)

In [15]:
log = LogisticRegression()

In [16]:
log.fit(X,Y['toxic'].values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
score(y['toxic'].values,log.predict(x))

Precision : 0.8895221394125383
Recall : 0.5379109225874867
Accuracy : 0.9499912265309703


In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

In [74]:
model = Sequential([
    Dense(16, input_shape=(X.shape[1],)),
    Activation('relu'),
    Dense(28,),
    Activation('relu'),
    Dense(32,),
    Dropout(0.3),
    Activation('relu'),
    Dense(2),
    Activation('softmax'),
])

In [75]:
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 6000
n = X.shape[0]

epochs = 2

for epoch in range(epochs):
    print (f"Epoch : {epoch}")
    for batch_index in range(batch_size,n,batch_size):
        X_ = X[batch_index-batch_size:batch_index]
        Y_ = Y.toxic.values[batch_index-batch_size:batch_index]
        model.fit(X_,Y_,epochs=2)

    X_ = X[batch_index:]
    Y_ = Y.toxic.values[batch_index:]
    model.fit(X_,Y_,)
        
notify.success()

Epoch : 0
Train on 6000 samples
Epoch 1/2
6000/6000 [==============================] - 6s 1ms/sample - loss: 0.3324 - acc: 0.9060
Epoch 2/2
6000/6000 [==============================] - 4s 677us/sample - loss: 0.1927 - acc: 0.9438
Train on 6000 samples
Epoch 1/2
6000/6000 [==============================] - 6s 1ms/sample - loss: 0.1849 - acc: 0.9528
Epoch 2/2
6000/6000 [==============================] - 4s 654us/sample - loss: 0.1124 - acc: 0.9685
Train on 6000 samples
Epoch 1/2
6000/6000 [==============================] - 6s 1ms/sample - loss: 0.2095 - acc: 0.9485
Epoch 2/2
6000/6000 [==============================] - 4s 663us/sample - loss: 0.1362 - acc: 0.9608
Train on 6000 samples
Epoch 1/2
6000/6000 [==============================] - 6s 1ms/sample - loss: 0.1861 - acc: 0.9528
Epoch 2/2
6000/6000 [==============================] - 4s 655us/sample - loss: 0.1350 - acc: 0.9652
Train on 6000 samples
Epoch 1/2
6000/6000 [==============================] - 6s 1ms/sample - loss: 0.1992 - ac

6000/6000 [==============================] - 4s 677us/sample - loss: 0.1235 - acc: 0.9683
Train on 6000 samples
Epoch 1/2
6000/6000 [==============================] - 6s 1ms/sample - loss: 0.1925 - acc: 0.9663
Epoch 2/2
6000/6000 [==============================] - 4s 682us/sample - loss: 0.1496 - acc: 0.9705
Train on 5678 samples
5678/5678 [==============================] - 6s 1ms/sample - loss: 0.2319 - acc: 0.9630


In [89]:
y__pred = [model.predict_classes(i.toarray()) for i in tqdm(x)]

39893it [02:40, 249.05it/s]


In [95]:
score(y.toxic.values.reshape(-1,1),y__pred)

Precision : 0.8414752507279197
Recall : 0.6895546129374337
Accuracy : 0.9583636226906976
